## Definiowanie problemu

W tym pliku rozwiązywane będzie zagadnienie rotacji klastrów. 

Wszystkie klastry znajdują się w pliku all.csv

Jak wiadomo przyłączam do nich CO2 poprzez przyłączanie go zawsze w tym miejscu. A przecież specyficznych miejsc, do których do przyłączenie może nastąpić jest bardzo dużo (i są opisane w all.csv) 

Dlatego propozycja poradzenia sobie z tym to rotowanie klastra, a nie CO2

Pomysł na rozwiązanie tego problemu jest następujący:

Wiadomo, że CO2 przyłącza się do atomów, które znajdują się na 1 i 2 (liczenie informatyczne) pozycji w liście.

Dlatego będę pobierać informacje o możliwym połączeniu np. do atomu 2 i 3 i będę szukał takiej rotacji (przy pomocy już bardzo dawno stworzonej listy rotacji) gdzie na 1 i 2 pozycji listy są atomy 2 i 3 (albo 3,2 jeżeli to będzie miało znaczenie, ale o tym może nieco później)

In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML
import random
import os

display(HTML("<style>.container { width:80% !important; }</style>"))

def hide(for_next=False, t = 'Toggle show/hide'):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = t
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ''
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)


os.chdir('/home/all2/Desktop/Inżynier/')
df = pd.read_csv('all.csv')
df.head()
hide(t='setup and import stuff')

In [2]:
hide(for_next=True, t= 'def corners(edges)_to_list(string)')

In [3]:
#na samym początku trzeba napisać stosowane funkcje

#ta funkcja zmienia topologię spowrotem na listę

def topology_to_list(string):
    
    string_from_topology = []
    for i in range(2,80,6):
        string_from_topology.append(string[i:i+2])  
        
    return string_from_topology

def mult24(n):
    for i in range(len(n)):
        for j in range(3):
            n[i][j] = float(n[i][j])*24
    return n

#ta funkcja będzie wyciągać informacje o połączeniach do narożników - warto tutaj
#zwrócić uwagę, że tak naprawdę potrzebujemy tylko informacji o tym, do którego atomu
#to przyłączenie ma nastąpić

def corners_to_list(string):
    
    string_from_corners = []
    
    for i in range(1,len(string)):
        
        #to jest calkiem smart -> sprawdzam czy przed sprawdzanym znakiem występuje znak
        #sugerujacy ze lista wlasnie sie otworzyla, co eliminuje wszystkie informacje 
        #liczbowe o sasiadach
        
        #potem sprawdzam czy po tej liczbie jest przecinek zeby wiedziec czy jest to
        #liczba jednocyfrowa czy dwucyfrowa
        #ten ostatni warunek eliminuje dodawanie do listy tego pierwszego nawiasu
        #i ogolnie ma tu byc bo troche sie to wykrzacza jak go nie ma
        
        if string[i-1] == '[' and string[i+1] == ',' and string[i] in  ['1','2','3','4','5','6','7','8','9']:
            string_from_corners.append(string[i])  
        elif string[i-1] == '[' and string[i] in  ['1']:
            string_from_corners.append(string[i] + string[i+1])
    return string_from_corners

#jak zawsze funkcja robiaca to samo dla polaczen krawedziowych bedzie dzialac tak samo
#tylko trudniej bedzie ja napisac

def edges_to_list(string):
    
    string_from_edges = []    
    
    for i in range(len(string)):
    
        single_connection = []
    
    #dołączanie informacji o pierwszym atomie
        if string[i-1] == '[':
            if string[i+1] == ',':
                if string[i] in ['1','2','3','4','5','6','7','8','9']:
                    single_connection.append(string[i])           
            elif string[i] in ['1']:
                single_connection.append(string[i] + string[i+1])
    #drugi atom chyba będę trackować po prostu po pozycji w liscie
    
        if len(single_connection) != 0:
            
            for j in range(i,i+20):
                
                #to są drugie litery Cu i Ni
                if string[j] in ['u','i']:
                    
                    #tutaj sprawdzam czy drugi atom jest jedno czy dwucyfrowy:                    
                    #i na wszelki wypadek breakuje jakby nie daj boże te +20 to było za dużo
                    
                    if string[j+5] == ',':
                        single_connection.append(string[j+4])
                        string_from_edges.append(single_connection)                        
                        break
                    else:
                        single_connection.append(string[j+4]+string[j+5])
                        string_from_edges.append(single_connection)
                        break
    return string_from_edges

#przydatna też będzie lista rotacji



In [4]:
hide(for_next=True, t= 'lista rotacji')

In [5]:
rotations = [
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
[0, 1, 3, 4, 5, 6, 2, 8, 9, 10, 11, 7, 12], 
[0, 1, 4, 5, 6, 2, 3, 9, 10, 11, 7, 8, 12], 
[0, 1, 5, 6, 2, 3, 4, 10, 11, 7, 8, 9, 12], 
[0, 1, 6, 2, 3, 4, 5, 11, 7, 8, 9, 10, 12],

[0, 2, 7, 8, 3, 1, 6, 11, 12, 9, 4, 5, 10],
[0, 2, 6, 7, 8, 3, 1, 5, 11, 12, 9, 4, 10], 
[0, 2, 1, 6, 7, 8, 3, 4, 5, 11, 12, 9, 10],
[0, 2, 3, 1, 6, 7, 8, 9, 4, 5, 11, 12, 10],
[0, 2, 8, 3, 1, 6, 7, 12, 9, 4, 5, 11, 10],
    
[0, 3, 1, 2, 8, 9, 4, 5, 6, 7, 12, 10, 11],
[0, 3, 2, 8, 9, 4, 1, 6, 7, 12, 10, 5, 11], 
[0, 3, 4, 1, 2, 8, 9, 10, 5, 6, 7, 12, 11], 
[0, 3, 8, 9, 4, 1, 2, 7, 12, 10, 5, 6, 11], 
[0, 3, 9, 4, 1, 2, 8, 12, 10, 5, 6, 7, 11],
    
[0, 4, 1, 3, 9, 10, 5, 6, 2, 8, 12, 11, 7],
[0, 4, 3, 9, 10, 5, 1, 2, 8, 12, 11, 6, 7], 
[0, 4, 5, 1, 3, 9, 10, 11, 6, 2, 8, 12, 7],
[0, 4, 9, 10, 5, 1, 3, 8, 12, 11, 6, 2, 7],
[0, 4, 10, 5, 1, 3, 9, 12, 11, 6, 2, 8, 7],

[0, 5, 1, 4, 10, 11, 6, 2, 3, 9, 12, 7, 8],
[0, 5, 4, 10, 11, 6, 1, 3, 9, 12, 7, 2, 8],
[0, 5, 6, 1, 4, 10, 11, 7, 2, 3, 9, 12, 8], 
[0, 5, 10, 11, 6, 1, 4, 9, 12, 7, 2, 3, 8], 
[0, 5, 11, 6, 1, 4, 10, 12, 7, 2, 3, 9, 8],

[0, 6, 1, 5, 11, 7, 2, 3, 4, 10, 12, 8, 9],
[0, 6, 2, 1, 5, 11, 7, 8, 3, 4, 10, 12, 9], 
[0, 6, 5, 11, 7, 2, 1, 4, 10, 12, 8, 3, 9],
[0, 6, 7, 2, 1, 5, 11, 12, 8, 3, 4, 10, 9], 
[0, 6, 11, 7, 2, 1, 5, 10, 12, 8, 3, 4, 9],
    
[0, 7, 2, 6, 11, 12, 8, 3, 1, 5, 10, 9, 4],
[0, 7, 6, 11, 12, 8, 2, 1, 5, 10, 9, 3, 4], 
[0, 7, 8, 2, 6, 11, 12, 9, 3, 1, 5, 10, 4], 
[0, 7, 11, 12, 8, 2, 6, 5, 10, 9, 3, 1, 4], 
[0, 7, 12, 8, 2, 6, 11, 10, 9, 3, 1, 5, 4],
    
[0, 8, 2, 7, 12, 9, 3, 1, 6, 11, 10, 4, 5],
[0, 8, 3, 2, 7, 12, 9, 4, 1, 6, 11, 10, 5], 
[0, 8, 7, 12, 9, 3, 2, 6, 11, 10, 4, 1, 5], 
[0, 8, 9, 3, 2, 7, 12, 10, 4, 1, 6, 11, 5], 
[0, 8, 12, 9, 3, 2, 7, 11, 10, 4, 1, 6, 5],

[0, 9, 3, 8, 12, 10, 4, 1, 2, 7, 11, 5, 6],
[0, 9, 4, 3, 8, 12, 10, 5, 1, 2, 7, 11, 6], 
[0, 9, 8, 12, 10, 4, 3, 2, 7, 11, 5, 1, 6], 
[0, 9, 10, 4, 3, 8, 12, 11, 5, 1, 2, 7, 6], 
[0, 9, 12, 10, 4, 3, 8, 7, 11, 5, 1, 2, 6],

[0, 10, 4, 9, 12, 11, 5, 1, 3, 8, 7, 6, 2],
[0, 10, 5, 4, 9, 12, 11, 6, 1, 3, 8, 7, 2], 
[0, 10, 9, 12, 11, 5, 4, 3, 8, 7, 6, 1, 2], 
[0, 10, 11, 5, 4, 9, 12, 7, 6, 1, 3, 8, 2], 
[0, 10, 12, 11, 5, 4, 9, 8, 7, 6, 1, 3, 2],

[0, 11, 5, 10, 12, 7, 6, 1, 4, 9, 8, 2, 3],
[0, 11, 6, 5, 10, 12, 7, 2, 1, 4, 9, 8, 3], 
[0, 11, 7, 6, 5, 10, 12, 8, 2, 1, 4, 9, 3], 
[0, 11, 10, 12, 7, 6, 5, 4, 9, 8, 2, 1, 3], 
[0, 11, 12, 7, 6, 5, 10, 9, 8, 2, 1, 4, 3],

[0, 12, 10, 9, 8, 7, 11, 5, 4, 3, 2, 6, 1],
[0, 12, 9, 8, 7, 11, 10, 4, 3, 2, 6, 5, 1], 
[0, 12, 7, 11, 10, 9, 8, 2, 6, 5, 4, 3, 1], 
[0, 12, 8, 7, 11, 10, 9, 3, 2, 6, 5, 4, 1],
[0, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1],
]

In [6]:
#dowody na to, że funkcje działają

print(((df['narożniki'][135])))
print()
print((corners_to_list(df['narożniki'][135])))

print(((df['krawędzie'][135])))
print()
print((edges_to_list(df['krawędzie'][135])))

[[1, 2.1, 3, 'Cu'], [2, 3, 2, 'Cu'], [4, 2, 3, 'Ni'], [5, 4, 1, 'Cu'], [7, 4, 1, 'Ni'], [8, 3, 2, 'Ni'], [12, 5, 0, 'Ni']]

['1', '2', '4', '5', '7', '8', '12']
[[1, 2.1, 3, 'Cu', 2, 3, 2, 'Cu'], [1, 2.1, 3, 'Cu', 4, 2, 3, 'Ni'], [1, 2.1, 3, 'Cu', 5, 4, 1, 'Cu'], [2, 3, 2, 'Cu', 7, 4, 1, 'Ni'], [2, 3, 2, 'Cu', 6, 2, 3, 'Ni'], [2, 3, 2, 'Cu', 3, 3, 2, 'Cu'], [2, 3, 2, 'Cu', 8, 3, 2, 'Ni'], [4, 2, 3, 'Ni', 5, 4, 1, 'Cu'], [4, 2, 3, 'Ni', 9, 4, 1, 'Ni'], [5, 4, 1, 'Cu', 10, 4, 1, 'Ni'], [7, 4, 1, 'Ni', 8, 3, 2, 'Ni'], [7, 4, 1, 'Ni', 11, 4, 1, 'Ni'], [7, 4, 1, 'Ni', 12, 5, 0, 'Ni'], [8, 3, 2, 'Ni', 12, 5, 0, 'Ni']]

[['1', '2'], ['1', '4'], ['1', '5'], ['2', '7'], ['2', '6'], ['2', '3'], ['2', '8'], ['4', '5'], ['4', '9'], ['5', '10'], ['7', '8'], ['7', '11'], ['7', '12'], ['8', '12']]


### Właściwy program

In [7]:
hide(for_next = True, t=' def rotacja(nr_klastra); def rotation_to_cuni(rotacja, nr_klastra)')

In [8]:
def rotacja(nr_klastra):
    
    edges = edges_to_list(df['krawędzie'][nr_klastra])
    corners = corners_to_list(df['narożniki'][nr_klastra])
    
    combinations = edges + corners
    
    r_corners = []
    r_edges = []
    r_mirror_edges = []
    for configuration in combinations:
              
        if isinstance(configuration,list):
            
            #w przypadku przylaczen do dwoch atomow trzeba znalezc taka rotacje, ktora
            #na 1 i 2 pozycji ma 1 i 2 atom, oraz taki, ktory na 1 i 2 pozycji ma 2 i 1 atom 
            # (bo niektore przypadki sa inne z tego wzgledu)
            for rotation in rotations:
                if rotation[1] == int(configuration[0]) and rotation[2] == int(configuration[1]):
                    r_edges.append(rotation)
                elif rotation[1] == int(configuration[1]) and rotation[2] == int(configuration[0]):
                    r_mirror_edges.append(rotation)
        else:
            
            #w przypadku przylaczen do jednego atomu trzeba znalezc taką rotacje
            #która na 1 pozycji (liczenie informatyczne) ma atom wskazany przez rotacje
            #zakladam w tym miejscu, ze wystarczy sprawdzic jedna rotacje, ktora spelnia 
            # ten jeden warunek co do tego jednego atomu
            
            for rotation in rotations:
                if rotation[1] == int(configuration):
                    r_corners.append(rotation)
                    #z racji na to, że chcemy uwzględniać każde możliwe ustawienie klastra
                    #które spełnia wymogi, a w przypadkach z przyłączaniem jednego atomu
                    #mówimy o 5 rotacjach, które pasują, tutaj nie ma break, jeżeli się 
                    #je dopisze to będzie tylko jedna opcja
                    
    #sekcja printujaca 
#     print('possible rotations:')
#     for entry in combinations:
#         print(entry)
#     print()
#     print('edges: ')    
#     for entry in r_edges:
#         print(entry)
#     print()
#     print('edges mirrors: ')
#     for entry in r_mirror_edges:
#         print(entry)
#     print()    
#     print('corners: ')
#     for entry in r_corners:
#         print(entry)
    
    
    return r_edges, r_mirror_edges, r_corners, #combinations

#ok po ten funkcji mamy wszystkie rotacje ktore chcemy zbadac w postaci numerycznej
#teraz trzeba zamienic je spowrotem na układ Cu, Ni

def rotation_to_cuni(nr_klastra, rotacja):
    
    topologia = topology_to_list(df['topologia'][nr_klastra])
#     print(topologia)
    pozycje_niklu = []
    wynik = []
    for i in range(len(topologia)):
        if topologia[i] == 'Ni':
            pozycje_niklu.append(i)
    for pos in rotacja:        
        if pos in pozycje_niklu:
            wynik.append('Ni') 
        else:
            wynik.append('Cu') 
#ostatnim krokiem będzie sprawdzanie czy w wyniku takiej rotacji nie powstał klaster, który
#jest bezrotacyjnym innym klastrem o takiej samej liczbie nikli
#można to sprawdzić po prostu porównując wszystkie topologie
#co bardzo ciekawe testy wskazały, że żadna rotacja nie powtarza się w ten sposób
#żadna rotacja też nie jest taka sama jak już wygenerowana rotacja danego rodzaju
    
    licznik = 0
    for cluster in range(len(df)):
        if str(wynik) == df['topologia'][cluster]:
            return 'w wyniku rotacji ' +str(rotacja[1:3]) +' otrzymano topologie klastra nr ' + str(cluster)
            break
        else:
            licznik +=1
    
    if licznik == 192:
        
        return 'w wyniku rotacji ' +str(rotacja[1:3]) + ' powstał nowy układ ' + str(wynik), wynik
    

In [9]:
hide(for_next= True, t='def all_non_repetitive_rotations(nr_klastra, edges, m_edges, corners)')

In [10]:
def all_non_repetitive_rotations(nr_klastra, edges, m_edges, corners):
    
    possible_rotation = rotacja(nr_klastra)
    all_new_clusters = []
    all_new_corners_clusters = []
    all_new_edges_clusters = []
    all_new_m_edges_clusters = []
    
    if corners == 1:
        print()
        print('narożnikowe przyłączenia')
        print()
        corner_rotation = possible_rotation[2]
        for i in range(len(corner_rotation)):
            cluster = rotation_to_cuni(nr_klastra, corner_rotation[i])
            if len(cluster) == 2:
                print(cluster[0])
                if cluster[1] not in all_new_corners_clusters:
                    all_new_corners_clusters.append(cluster[1])
            else:
                print(cluster)
        all_new_clusters.append(all_new_corners_clusters)
                
    if edges == 1:
        print()
        print('krawędziowe przyłączenia')
        print()
        edge_rotation = possible_rotation[0]
        for i in range(len(edge_rotation)):
            cluster = rotation_to_cuni(nr_klastra, edge_rotation[i])
            if len(cluster) == 2:
                print(cluster[0])
                if cluster[1] not in all_new_edges_clusters:
                    all_new_edges_clusters.append(cluster[1])
            else:
                print(cluster)
        all_new_clusters.append(all_new_edges_clusters)
    if m_edges == 1:
        print()
        print('krawędziowe lustrzane przyłączenia')
        print()
        m_edge_rotation = possible_rotation[1]
        for i in range(len(m_edge_rotation)):
            cluster = rotation_to_cuni(nr_klastra, m_edge_rotation[i])
            if len(cluster) == 2:
                print(cluster[0])
                if cluster[1] not in all_new_m_edges_clusters:
                    all_new_m_edges_clusters.append(cluster[1])
            else:
                print(cluster)
        all_new_clusters.append(all_new_m_edges_clusters)
        return ((all_new_clusters))
    


BARDZO WAŻNA SPRAWA
program w tym momencie działa tak, ze jest zwracana pierwsza pasująca rotacja
i o ile w przypadku przyłączeń krawędziowych tylko jedna jest słuszna, bo 1 i 2 miejsce (liczenie informatyczne) jest zawsze jedno w podanej konfiguracji
o tyle w przypadku narożników 1 miejsce (liczenie informatyczne) występuje w 5 rotacjach 
nie wiem czy warto to uwzględniać

In [11]:
results = []
licznik = 0
for i in range(192):
    lista = (all_non_repetitive_rotations(i,1,1,1))
    
    lenght = len(lista[0])+len(lista[1])+len(lista[2])
    licznik += lenght
    original_lenght = df['C/O/M-O-C/M-O-C-_'][i]+2*df['M-O-C-O/M-C-M'][i]
    
    
    if original_lenght>lenght+3:
        results.append('o ' + str(i))
    else:
        results.append('nic się nie powtarza: ' + str(i))




narożnikowe przyłączenia

w wyniku rotacji [1, 2] otrzymano topologie klastra nr 0
w wyniku rotacji [1, 3] otrzymano topologie klastra nr 0
w wyniku rotacji [1, 4] otrzymano topologie klastra nr 0
w wyniku rotacji [1, 5] otrzymano topologie klastra nr 0
w wyniku rotacji [1, 6] otrzymano topologie klastra nr 0

krawędziowe przyłączenia

w wyniku rotacji [1, 2] otrzymano topologie klastra nr 0

krawędziowe lustrzane przyłączenia

w wyniku rotacji [2, 1] otrzymano topologie klastra nr 0

narożnikowe przyłączenia

w wyniku rotacji [1, 2] otrzymano topologie klastra nr 1
w wyniku rotacji [1, 3] otrzymano topologie klastra nr 1
w wyniku rotacji [1, 4] otrzymano topologie klastra nr 1
w wyniku rotacji [1, 5] otrzymano topologie klastra nr 1
w wyniku rotacji [1, 6] otrzymano topologie klastra nr 1

krawędziowe przyłączenia

w wyniku rotacji [1, 2] otrzymano topologie klastra nr 1

krawędziowe lustrzane przyłączenia

w wyniku rotacji [2, 1] otrzymano topologie klastra nr 1

narożnikowe przyłąc

w wyniku rotacji [4, 1] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [4, 3] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [4, 5] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [4, 9] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu']
w wyniku rotacji [4, 10] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu']
w wyniku rotacji [9, 3] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [9, 4] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [9, 8] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu']
w wyniku rotacji [9, 10

w wyniku rotacji [11, 7] powstał nowy układ ['Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu']
w wyniku rotacji [12, 11] powstał nowy układ ['Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni']

narożnikowe przyłączenia

w wyniku rotacji [1, 2] otrzymano topologie klastra nr 12
w wyniku rotacji [1, 3] powstał nowy układ ['Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [1, 4] powstał nowy układ ['Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [1, 5] powstał nowy układ ['Cu', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu']
w wyniku rotacji [1, 6] powstał nowy układ ['Cu', 'Ni', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu']
w wyniku rotacji [3, 1] powstał nowy układ ['Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [3, 2] powstał nowy układ ['Cu', 'Cu', 'N

w wyniku rotacji [10, 9] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni']
w wyniku rotacji [10, 11] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni']
w wyniku rotacji [10, 12] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni']

krawędziowe przyłączenia

w wyniku rotacji [1, 2] otrzymano topologie klastra nr 15
w wyniku rotacji [1, 4] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [1, 5] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [4, 5] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu']
w wyniku rotacji [4, 10] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu']
w wyniku rotacji [5, 10] powstał nowy układ ['Ni', 'Cu'

w wyniku rotacji [7, 8] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni', 'Cu']
w wyniku rotacji [7, 11] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu']
w wyniku rotacji [7, 12] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu']
w wyniku rotacji [10, 4] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni']
w wyniku rotacji [10, 5] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Cu', 'Ni']
w wyniku rotacji [10, 9] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Ni']
w wyniku rotacji [10, 11] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Cu', 'Ni']
w wyniku rotacji [10, 12] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni']

krawędziowe pr

...

w wyniku rotacji [3, 8] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [5, 6] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni']
w wyniku rotacji [5, 10] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni']
w wyniku rotacji [5, 11] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [7, 8] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu']
w wyniku rotacji [7, 11] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu']
w wyniku rotacji [7, 12] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu']
w wyniku rotacji [11, 12] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Cu', 'Ni']

krawędziowe lustr

w wyniku rotacji [1, 6] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu']
w wyniku rotacji [3, 4] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni']
w wyniku rotacji [3, 8] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [4, 5] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni']
w wyniku rotacji [4, 10] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni']
w wyniku rotacji [5, 6] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Cu', 'Ni']
w wyniku rotacji [5, 11] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni']

krawędziowe lustrzane przyłączenia

w wyniku rotacji [2, 1] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu

w wyniku rotacji [1, 2] otrzymano topologie klastra nr 180
w wyniku rotacji [1, 4] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [1, 5] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [4, 5] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [4, 10] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni']
w wyniku rotacji [5, 10] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni']
w wyniku rotacji [10, 11] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Cu']

krawędziowe lustrzane przyłączenia

w wyniku rotacji [2, 1] powstał nowy układ ['Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [4, 1] powstał nowy układ ['N

w wyniku rotacji [1, 2] otrzymano topologie klastra nr 183
w wyniku rotacji [1, 3] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [1, 4] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [1, 5] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [1, 6] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni']
w wyniku rotacji [3, 1] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni']
w wyniku rotacji [3, 2] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni']
w wyniku rotacji [3, 4] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [3, 8] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Cu', '

w wyniku rotacji [1, 3] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni']
w wyniku rotacji [1, 4] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni']
w wyniku rotacji [1, 5] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [1, 6] powstał nowy układ ['Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [2, 7] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [2, 6] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [2, 1] powstał nowy układ ['Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [2, 3] powstał nowy układ ['Ni', 'Ni', 'Ni', 'Cu', 'Ni', 'Cu', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']
w wyniku rotacji [2, 8] 

## Tworzenie inputów

In [36]:
#ok, co my tak wlasciwie chcemy zrobic, jakie są różnice?
#przypadki przyłączenia jednego atomu, czyli MO, MOC, MC, to lista 3 (liczenie normalne)
#inne przypadki to lista 1
#lista druga to sytuacja, w której w danej rotacji są zamienione dwa pierwsze atomy 
#DOTYCZY TO TYLKO SYTUACJI, GDY TE ATOMY TO Cu i Ni oraz analizujemy połączenie, które
#różni się z tego względu, ale teraz widzę, że to chyba nie jest konieczne, bo przecież
#jedyny taki przypadek MOCM z wodorem - a ja po prostu rozróżniam przypadki MOCM_H_bond i
#MOCM_H_n_bond

#w jaki sposob ja tak wlasciwie chce generowac wspolrzedne dodawanego CO2?
#napewno średnia ważona ale czego?
#jeżeli chciałbym do każdego przypadku generować optymalizację czysto niklową i czysto Cu
#to to sie mija z celem
#chyba ze po prostu wygeneruje to na podstawie jednego klastra albo tych dwóch które mam
#argumentując, że nie mam czasu, żeby implementować lepsze rozwiązanie

#polaczenie MOC_ jest analizowane ze wzgleduy na dwa atomy a nie 3
for CO2 in added_particle:
    print(CO2[0], len(CO2[1]))

[['MC',
  [[4.572, 9.5835, 3.4444],
   [3.5577, 9.9961, 3.9862],
   [5.094, 10.2272, 2.3462],
   [4.4973, 10.9938, 2.1699]]],
 ['MCMH',
  [[6.47397, 9.1645, 2.32497],
   [5.28787, 10.02869, 2.10069],
   [7.227, 9.25122, 1.07235],
   [7.04164, 9.95357, 0.44533]]],
 ['MCMH_p',
  [[6.38575, 9.16684, 2.18302],
   [6.82338, 10.42408, 2.47105],
   [5.45309, 9.06317, 1.10022],
   [6.53007, 11.0628, 1.75903]]],
 ['MO',
  [[4.20267, 10.39705, 2.99179],
   [4.81122, 9.34572, 3.43126],
   [3.49797, 11.61166, 2.48555]]],
 ['MOC',
  [[4.49699, 9.66461, 3.89761],
   [4.40781, 9.41209, 2.67005],
   [3.97995, 10.64996, 4.56887]]],
 ['MOC_',
  [[6.26608, 8.65327, 2.02836],
   [6.5026, 9.93047, 2.05334],
   [5.14173, 7.50164, 1.36368]]],
 ['MOCM',
  [[7.02313, 8.98643, 1.89168],
   [5.779, 9.49935, 2.55575],
   [7.49132, 9.6426, 0.87176]]],
 ['MOCM_H_bond',
  [[6.85117, 8.84647, 1.91352],
   [5.63397, 9.19891, 2.48515],
   [7.31031, 9.28968, 0.70909],
   [4.93058, 9.88011, 2.1156]]],
 ['MOCM_H_n_bond',


In [41]:
#MC, MO --> jednoatomowe
#MOC, MOC_, MOCM, MOCM_H_bond, MOCM_H_n_bond, MOCOM, MCMH, MCMH_p, MCM_p --> dwuatomowe

generating_cluster = [191]
generating_particle =['MC','MCMH','MCMH_p','MOC','MOC_','MOCM','MOCM_H_bond','MOCM_H_n_bond','MCM_p']

#jeżeli chcemy generować inputy do pelnej optymalizacji to wpisujemy opt, jak single
#pointy to sp
sp_or_opt = 'opt'
rot_input = 'n'
import os
import shutil

#jeżeli program nie znajduje się w folderze klastry to lokalizacja jest zmieniana właśnie na ten folder
if os.getcwd() != '/home/all2/Desktop/Inżynier/klastry':
    os.chdir('/home/all2/Desktop/Inżynier/klastry')
    
#wyznaczam teraz listę rotacji, które będę sprawdzać

for particle in generating_particle:
    for cluster in generating_cluster:
        if particle in ['MC','MO','MOC']:
            possible_rotations = rotacja(cluster)[2]
            corner_or_edge = 'c'
        else:
            possible_rotations = rotacja(cluster)[0]
            corner_or_edge = 'e'
            
        #ok mamy teraz listę rotacji dla jednego klastra
        #teraz trzeba ją zamienić na topologię, na szczescie mam juz do tego funkcje
        
        possible_rotations_cuni = []
        for i in range(len(possible_rotations)):
            #biore pierwszy element tej funkcji, gdyz zerowy to jakies opisy
            possible_rotations_cuni.append(rotation_to_cuni(cluster,possible_rotations[i])[1])
            #jeżeli w wyniku rotacji powstaje topologia, która już istnieje, to 
            #zwracana jest pusta lista --> trzeba usunąć puste listy
        
        #jakby tego było mało, to program działa tak, że odrzuca oryginalne rotacje jako
        #powtarzalne, wiec chyba najlatwiej je będzie wrzucić ręcznie o tutaj
        clean_possible_rotations_cuni = []
        clean_possible_rotations_cuni.append(topology_to_list(df['topologia'][cluster]))
        
        #to jest lista, ktora przechowuje informacje jakie rotacje przechodza dalej
        #jest wykorzystywana chwile pozniej do nazywania folderow
        
        list_of_passed_rotation = [0]
        
        for i in range(len(possible_rotations_cuni)):
            if len(possible_rotations_cuni[i]) ==13:
                clean_possible_rotations_cuni.append(possible_rotations_cuni[i])
                list_of_passed_rotation.append(i)
        #ok jesteśmy na etapie, w którym mamy listę, która zawiera wszystkie topologie 
        #do sprawdzenia w obrębie jednego klastra, w obrębie jednego typu przyłączenia
        
        #teraz przypisuje koordynaty do zmiennej position_of_particle
        
        if sp_or_opt == 'sp':
            added_particle = cord_of_particle(cluster, particle)
            if len(added_particle) == 4:
                hydrogen = 1
            else:
                hydrogen = 0
        elif sp_or_opt == 'opt':
            for position_in_list in range(len(added_particle)):
                if added_particle[position_in_list][0] == particle:
                    position_of_particle = position_in_list 
                    
            #muszę sprawdzić czy przyłączana cząsteczka ma w sobie wodór czy nie, żeby zrobić dobrego POTCARA
            if (len(added_particle[position_of_particle][1]) == 4):
                hydrogen = 1
            else:
                hydrogen = 0
        #reszta działań będzię przebiegać w kolejnej pętli
        
        #uwzgledniam czy chce generować wszystkie rotacje czy tylko pierwsze z brzegu
        if rot_input == 'y':
            range_of_for = len(clean_possible_rotations_cuni)
        elif rot_input == 'n':
            range_of_for = 1
            
        for rot in range(range_of_for):
        #tworze folder o nazwie "numer klastra + typ przyłączenia czasteki + 1 i 2 atom"       
        
            name_of_dir = str(cluster) + '_' + particle + '_' +str(possible_rotations[list_of_passed_rotation[rot]][1])+ '_'+str(possible_rotations[list_of_passed_rotation[rot]][2]) + corner_or_edge           
            os.mkdir(name_of_dir) 
            # kopiuje do niego INCARA
            shutil.copyfile(r'INCAR' , r'/home/all2/Desktop/Inżynier/klastry/' +name_of_dir +'/INCAR')
            #teraz robię POTCARA
            os.chdir('/home/all2/Desktop/Inżynier/klastry/PSEUDO/Cu')
            with open('POTCAR','r') as file:
                lines_Cu = file.readlines()
            os.chdir('/home/all2/Desktop/Inżynier/klastry/PSEUDO/Ni')
            with open('POTCAR','r') as file:
                lines_Ni = file.readlines()
            os.chdir('/home/all2/Desktop/Inżynier/klastry/PSEUDO/C')
            with open('POTCAR','r') as file:
                lines_C = file.readlines()
            os.chdir('/home/all2/Desktop/Inżynier/klastry/PSEUDO/O')
            with open('POTCAR','r') as file:
                lines_O = file.readlines()

            if hydrogen ==1:
                os.chdir('/home/all2/Desktop/Inżynier/klastry/PSEUDO/H')
                with open('POTCAR','r') as file:
                    lines_H = file.readlines()                   
                lines_potcar = lines_Cu + lines_Ni+lines_C+lines_O+lines_H
            else:
                lines_potcar = lines_Cu + lines_Ni+lines_C+lines_O

            os.chdir('/home/all2/Desktop/Inżynier/klastry/' + name_of_dir)    
            with open('POTCAR','w') as file:
                for line in lines_potcar:
                    print(line.rstrip('\n'), file = file)

            os.chdir('/home/all2/Desktop/Inżynier/klastry')

            #przenosze do tego folderu POTCAR
            #shutil.copyfile(r'C:\\Users\\stott\\Desktop\\all2\\Studia\\Inżynier\\klastry\\POTCAR' , r'C:\\Users\\stott\\Desktop\\all2\\Studia\\Inżynier\\klastry\\' +name_of_dir +'\\POTCAR')

            #teraz tam wchodze żeby zrobić POSCAR
            os.chdir('/home/all2/Desktop/Inżynier/klastry/' +name_of_dir)       

            #stworzenie dwóch list pomocnicznych przechowujących informacje o współrzędnych atomów - koniecznie aby ułożyć je po kolei
            lista_Ni = []
            lista_Cu = []

            
            klaster = clean_possible_rotations_cuni[rot]

            #generuję w locie plik POSCAR
            with open('POSCAR','w+') as plik: 

                #drukuje w nim co trzeba 
                if sp_or_opt == 'opt':                    
                    print( str(cluster)+' , ' +' '.join(klaster) + ' , ' + str(added_particle[position_of_particle][0]), file=plik)
                elif sp_or_opt == 'sp':
                    print( str(cluster)+' , ' +' '.join(klaster) + ' , ' + particle, file=plik)

                print('   1.00000000000000', file=plik)        
                print('    24    0.0000000000000000    0.0000000000000000', file = plik)        
                print('    0.0000000000000000   24    0.0000000000000000', file = plik)
                print('    0.0000000000000000    0.0000000000000000   24', file = plik)

                if hydrogen == 1:                
                    print('Cu Ni C O H',file = plik)
                else:
                    print('Cu Ni C O',file = plik)
                if hydrogen == 1:
                    print(' ' + str((13-df['liczba Ni'][cluster]))+' ' +str(df['liczba Ni'][cluster]) + ' 1 2 1', file= plik)
                else:
                    print(' ' + str((13-df['liczba Ni'][cluster]))+' ' +str(df['liczba Ni'][cluster]) + ' 1 2', file= plik)

                print('Cartesian',file=plik)        

                #korzystając z załadowanej topologii i listy koordynatów generuję stringi bardzo podobne do kartezjańskiego inputu w ORCA        
                for j in range(len(klaster)):            
                    atom = (str(klaster[j]) +'     ' + str(cluster_cor[j][0])+'    ' + str(cluster_cor[j][1])+'    ' + str(cluster_cor[j][2]))             

                    #dokonuję segregacji tych stringów na dwie listy - atomów miedzi i atomów niklu
                    if atom[0:2] == 'Cu':
                        lista_Cu.append(atom)
                    else:
                        lista_Ni.append(atom)

                    #wypisuję w POSCARZE najpierw wszystkie koordynaty miedzi
                for k in range(len(lista_Cu)):
                    print(lista_Cu[k][6:],file=plik)

                    #a potem wszystkie koordynaty niklu
                for k in range(len(lista_Ni)):
                    print(lista_Ni[k][6:],file=plik)     

                #drukowanie czastek

                #zakres tej petli to jest chyba najbardziej clever i out of the box rzecz jaką dzisiaj zrobiłem
                for k in range(3+hydrogen):
                    if sp_or_opt == 'opt':
                        print(' ' +str(added_particle[position_of_particle][1][k][0]) + ' ' +str(added_particle[position_of_particle][1][k][1]) + ' ' +str(added_particle[position_of_particle][1][k][2]), file = plik)   
                    elif sp_or_opt == 'sp':
                        print(' ' + str(added_particle[k][0])+ ' ' + str(added_particle[k][1]) + ' '+ str(added_particle[k][2]), file = plik)
            #wracam do folderu klastry
            os.chdir('/home/all2/Desktop/Inżynier/klastry')


             
             



In [ ]:
hide(for_next= True, t = 'zbiór koordynatów wykorzystywany do tworzenia inputów')

In [34]:
cluster_cor =[
[6.5496685184414583, 6.033443781138531, 4.400021141755959],
[5.5133159295396688, 8.15014172053236, 3.9219593800387678],
[7.5847823276760713, 7.3817913195743525, 2.699631552138487],
[5.394168510816928, 6.217971597652884, 2.2911885257897637],
[4.1515243349877448, 6.090263868390125, 4.483845887337588],
[5.5732855995491968, 7.199141367109155, 6.256367874602404],
[7.7136999646584545, 7.987329095526762, 5.165874086788361],
[8.947766264342306, 5.97448383770136, 4.310807579100965],
[7.523709042140615, 4.8771157300215595, 2.5364893365934247],
[5.3946324797117775, 4.076073687679214, 3.628981203610625],
[5.509710575776312, 4.684201449094534, 6.0937568175929275],
[7.7011747759426434, 5.854957765598119, 6.508647995201448],
[7.5871416764168993, 3.918974779981121, 4.879498619449174]
]

#before aproximmation, wykorzystywane do generowania POSCAROW do pelnej optymalizacji
added_particle = [

#approved
['MC',
[[4.5720,9.5835,3.4444],
 [3.5577,9.9961,3.9862],
 [5.0940,10.2272,2.3462],
 [4.4973,10.9938,2.1699]]],

# do weryfikacji oddalic tlen laczacy sie z metalem, przyblizyc ten, ktory sie z nim nie laczy i zrobic nowy przypadek prostopadly do tego
    
['MCMH',
  [[6.47397,        9.16450,        2.32497],
  [5.28787,       10.02869,        2.10069],
  [7.22700,        9.25122,        1.07235],
  [7.04164,        9.95357,        0.44533]]],
       
#nowy przypadek do weryfikacji - MCMH ustawione prostopadle
['MCMH_p',
  [[6.38575,        9.16684,        2.18302],
  [6.82338,       10.42408,        2.47105],
  [5.45309,        9.06317,        1.10022],
  [6.53007,       11.06280,        1.75903]]],
   
#approved
['MO',
  [[4.20267,       10.39705,        2.99179],
  [4.81122,        9.34572,        3.43126],  
  [3.49797,       11.61166,        2.48555]]], 
    
#do weryfikacji - przesunać czasteczke tak zeby C-M bylo krotsze niz O-M 
  
['MOC',
 [[4.49699,       9.66461,        3.89761],
 [4.40781,        9.41209,        2.67005],
 [3.97995,       10.64996,        4.56887]]],   
  
#approved
['MOC_',
 [[6.26608,8.65327,2.02836],
  [6.50260,9.93047,2.05334],
  [5.14173,7.50164,1.36368]]],
    
#approved
['MOCM',
  [[7.02313,8.98643,1.89168],
  [5.77900,9.49935,2.55575],  
  [7.49132,9.64260,0.87176]]], 
    
#approved
['MOCM_H_bond',
  [[6.85117,       8.84647,       1.91352],
  [5.63397,      9.19891,       2.48515],  
  [7.31031,       9.28968,       0.70909],
  [4.93058,       9.88011,       2.11560  ]]], 
         
#approved
['MOCM_H_n_bond', 
  [[6.99341,       8.95106,       1.93994],
  [5.63984,       9.31627,       2.39413],   
  [7.29056,       9.25477,       0.67283],
  [6.75821,      10.06369,       0.49781]]],
       
#approved
['MOCOM',
  [[6.26604,        9.76804,        1.72506],
  [5.32390,        9.53111,        2.79566],  
  [7.33292,        8.67929,        1.70791],
  [6.07650,       10.65772,        1.06022]]],
    
#approved
['MCM_p',
[[6.61866,9.35538,2.42514],
[7.21966,10.44342,2.89625],
[5.77351,9.54820,1.33512]]]
  
]
#after aproximmation, wykorzystywane do generowania POSCAROW do SP

added_particle_Cu = [
    
    ['MO',
    [[3.9440264310564697, 10.955881178237659, 2.901765447243581],
    [4.548663976898266, 10.211680880589523, 3.591091844812752],
    [3.3439981221669095, 11.694120500340926, 2.214661464859685]]],
    
    ['MOCOM',
    [[6.334299413994003, 9.753218569902113, 1.8190350820146746],
    [5.4228921729940485, 9.635666313084968, 2.697152020349172],
    [7.30817848349694, 8.97114011735288, 1.5867353081706406],
    [6.271752823172631, 10.650070317533734, 1.1739475441069103]]]
    
]


added_particle_Ni = [
    
    ['MO',
    [[4.090790489692724, 10.670019068016806, 2.9429214571117894],
    [4.692659269718343, 9.784766191887272, 3.443967395040139],
    [3.49343054160858, 11.55532197352506, 2.4502530017750304]]],
    
    ['MOCOM',
    [[6.303150002644902, 9.686264507749085, 1.8133085450422033],
    [5.419485129389863, 9.581068781564856, 2.7266465646556273],
    [7.299121445614543, 8.915879054072333, 1.6162462367137391],
    [6.187170622761804, 10.533948341532327, 1.1128575640121567]]]
    
]

In [14]:
hide(for_next=True, t='def cord_of_particle(nr_klastra, typ_polaczenia)')

In [15]:
#ok teraz trzeba napisać funkcję, która będzie robić średnią ważoną z koordynatów

def cord_of_particle(nr_klastra, typ_polaczenia):
    
    #zczytuje liczbę nikli
    
    ni = df['liczba Ni'][nr_klastra]
    cu = 13 - ni
    
    if typ_polaczenia == 'MO':
        
        #zczytuje koordynaty czystych klastrów 
        ni_cord = clear_Ni_MO_opt[-3:]
        cu_cord = clear_Cu_MO_opt[-3:]   
        
    elif typ_polaczenia == 'MOCOM':
        
        ni_cord = clear_Ni_MOCOM_opt[-4:]
        cu_cord = clear_Cu_MOCOM_opt[-4:]
    
    #tu trzeba dopisać inne typy analogicznie jak te dwa    
    
    #no tutaj po prostu robie średnią ważoną 
    #jedynie ciekawą sprawą jest fakt, że final_cord zawiera pustego stringa
    #wynika to ze sposobu napisania kodu do generowania INPUTÓW
    #bo do generowania pełnych optymalizacji biore pod uwage dwuwymiarowa liste
    #gdzie pierwszym elementem jest string identyfikujacy typ przylaczenia
    #i tak odnosze sie tylko do drugiego elementu wiec tutaj po prostu ten pierwszy
    #jest pusty i tyle
    
    final_cord = []

        
    for particle in range(len(ni_cord)):
        particle_final_cord = []
        for coordinate in range(3):            
            particle_final_cord.append((ni*ni_cord[particle][coordinate]+cu*cu_cord[particle][coordinate])/13)
        final_cord.append(particle_final_cord)

    return final_cord

In [16]:
hide(for_next=True, t='lista zoptymalizowanych klastrów')

In [18]:
clear_Cu_MC_opt = mult24([
['0.2760534862878593', '0.2460868189288341', '0.1859028981926940'],
['0.2320386220827705', '0.3352640032542641', '0.1668627337863227'],
['0.3198528307798917', '0.3029967588344925', '0.1159493518862670'],
['0.2278475997080710', '0.2544200198491374', '0.0983061014100364'],
['0.1762018362703806', '0.2477068725177549', '0.1904339560920775'],
['0.2367549930352889', '0.2927043062438914', '0.2645913849238419'],
['0.3251027417953452', '0.3266533134174412', '0.2183218816511531'],
['0.3759551089261366', '0.2419648836047765', '0.1812482108791471'],
['0.3160951821443871', '0.1981104727424750', '0.1075744998670215'],
['0.2279699488134925', '0.1644366250808208', '0.1529474936018097'],
['0.2334153991010401', '0.1876131329810729', '0.2553283196687685'],
['0.3251250225572284', '0.2357613025324539', '0.2726361281812564'],
['0.3189089146118902', '0.1568437459081578', '0.2037604763851743'],
['0.1905008621937760', '0.3993145968061337', '0.1435195391163448'],
['0.1482387558215435', '0.4165081462971051', '0.1660948152986206'],
['0.2122538108947171', '0.4261333337563527', '0.0977591571925298'],
['0.1873903016428373', '0.4580783339115102', '0.0904155518669444']
])

clear_Ni_MC_opt = mult24([
['0.2753010335461732', '0.2464649537356242', '0.1860186937343266'],
['0.2311884896146526', '0.3349457717603740', '0.1668483796465358'],
['0.3150720709013719', '0.3027572580513076', '0.1208934237566225'],
['0.2273554483900100', '0.2548200203593832', '0.1025502466644650'],
['0.1797804921389874', '0.2469071783411464', '0.1912778983812034'],
['0.2387669629641573', '0.2901498695846877', '0.2615323772699215'],
['0.3226975682058428', '0.3244830785586525', '0.2190825061495060'],
['0.3708980463673431', '0.2433810284331123', '0.1819005781361936'],
['0.3129830477485459', '0.2010640144046560', '0.1112068750023743'],
['0.2288471583236271', '0.1665810896554561', '0.1531649369753122'],
['0.2367075849809617', '0.1890578137807727', '0.2513887636298651'],
['0.3239912049442408', '0.2359355908682015', '0.2703178356799114'],
['0.3189518019785106', '0.1586838003723576', '0.2025621041127738'],
['0.1942860683250724', '0.3975600976312968', '0.1407286016955991'],
['0.1524609667680967', '0.4126508153042943', '0.1662508222349481'],
['0.2131019039768881', '0.4267184103126440', '0.0952855309706091'],
['0.1873155674921739', '0.4584358755127078', '0.0906429259598423']
])

clear_Cu_MOC_opt = mult24([
['0.2721769081558024', '0.2500576855548353', '0.1836330440172053'],
['0.2218086678847657', '0.3375552579716899', '0.1663395293534474'],
['0.3119040859407636', '0.3081972491849132', '0.1121946380379286'],
['0.2170405948816607', '0.2541977175824465', '0.0998277522217047'],
['0.1724836789631525', '0.2453031506805616', '0.1931218746338336'],
['0.2317467565245538', '0.2931896747152714', '0.2646833791024696'],
['0.3155749562500132', '0.3335060439670260', '0.2184196434115220'],
['0.3714499001477733', '0.2541085660591956', '0.1759160130686998'],
['0.3112286958141159', '0.2055998776021835', '0.1032453610644542'],
['0.2281171552189494', '0.1650179030131356', '0.1522515115903758'],
['0.2359119100076209', '0.1898089983797485', '0.2535127125853225'],
['0.3248282850680010', '0.2430422582328751', '0.2691862582292721'],
['0.3213507049561181', '0.1633361124813815', '0.2008853231162465'],
['0.2440993245608999', '0.3921333321810512', '0.1069364360381133'],
['0.2126256595226502', '0.4320041454380237', '0.1048071376048820'],
['0.2880118827698226', '0.3789216102890013', '0.0795343859245352']  
])

clear_Ni_MOC_opt = mult24([
['0.2707582872624220', '0.2539997215949541', '0.1825579384465331'],
['0.2246267114808473', '0.3375311331124273', '0.1644022487654350'],
['0.3127049230643229', '0.3099955574677850', '0.1113655250414431'],
['0.2206622204073208', '0.2546282441192195', '0.1015840379066343'],
['0.1749135452619715', '0.2482208892425437', '0.1915620196483370'],
['0.2318771614450925', '0.2943337319091061', '0.2613780363875580'],
['0.3150865404418646', '0.3305174314751510', '0.2165650591265999'],
['0.3671168169353687', '0.2535932447616866', '0.1753361635545193'],
['0.3109840659373737', '0.2082766434653538', '0.1073662102523988'],
['0.2297190582978021', '0.1711538203567296', '0.1540692958239376'],
['0.2351744033322989', '0.1936145583419175', '0.2511764161942318'],
['0.3203055310733370', '0.2438146156372742', '0.2646979288190897'],
['0.3185314003133710', '0.1678728670018793', '0.1998041522003014'],
['0.2491012458294225', '0.3833620330077336', '0.1081068760840441'],
['0.2091426609448639', '0.4142995140511604', '0.1196342168106718'],
['0.2896545946389835', '0.3807655777884184', '0.0748888749382780']   
])

clear_Cu_MOCM_H_bond_opt = mult24([    
['0.2718143752714359', '0.2537194579338015', '0.1828805533113010'],
['0.2275014311652119', '0.3416327850964594', '0.1640372817043985'],
['0.3168873633476983', '0.3065915504284389', '0.1113709964189533'],
['0.2238499615370524', '0.2584184447177501', '0.0940280786092530'],
['0.1715689944292738', '0.2556471413658168', '0.1871638536362239'],
['0.2350294351834577', '0.3024771413151330', '0.2624249650761040'],
['0.3247010866195618', '0.3327301437686476', '0.2143620105577242'],
['0.3722502676049183', '0.2488854661215731', '0.1791953025565335'],
['0.3085533405793146', '0.2017391036267286', '0.1046265310878246'],
['0.2202014185289276', '0.1716797989171825', '0.1532084794554292'],
['0.2272850133124509', '0.1995359767844991', '0.2529661662150898'],
['0.3195969465746466', '0.2444184565930676', '0.2715528350746275'],
['0.3128071338041390', '0.1655462549608017', '0.2023638102701071'],
['0.3351894200250312', '0.3550194746811031', '0.0568832408910824'],
['0.2105369703540693', '0.4091565530592681', '0.1359560056924456'],
['0.3443790236342476', '0.3867260580879394', '0.0218472523701934'],
['0.1950394846952186', '0.4064749425417882', '0.0985372204060549']
])

clear_Ni_MOCM_H_bond_opt = mult24([    
['0.2696923245369072', '0.2555002929025370', '0.1809692895417458'],
['0.2184826908482295', '0.3402811299226315', '0.1659913172992692'],
['0.3062111920035342', '0.3126206993508887', '0.1107777062755061'],
['0.2172732065694092', '0.2583252263218613', '0.1018095661370720'],
['0.1742283788455589', '0.2488055740618077', '0.1918306333535447'],
['0.2310076648830863', '0.2944529380125669', '0.2592181184650925'],
['0.3114520292642408', '0.3342354021344701', '0.2131645466810423'],
['0.3659555357107043', '0.2577585677267344', '0.1756169888766814'],
['0.3072821175684516', '0.2100442826718136', '0.1060664589459480'],
['0.2278220502053094', '0.1725986455174978', '0.1545537357047808'],
['0.2343673755813771', '0.1946984728741586', '0.2526932767652147'],
['0.3197474734710092', '0.2471056231222959', '0.2637749306012657'],
['0.3177519005364373', '0.1712715551724888', '0.1988161952309672'],
['0.3365842006373898', '0.3487241814075975', '0.0563424762318126'],
['0.1991205159326050', '0.4044797587990248', '0.1364272570312021'],
['0.3569903974752052', '0.3727744925918280', '0.0192941406209315'],
['0.2232226125972002', '0.4167219074097956', '0.1060579455712703']
])

clear_Cu_MOCOM_opt = mult24([    
['0.2717862719261115', '0.2514427030464020', '0.1825179082495662'],
['0.2226683856001294', '0.3386844602024912', '0.1630248601087719'],
['0.3155059647723644', '0.3061163411898375', '0.1085954505838774'],
['0.2208393679320406', '0.2521544470849880', '0.0968938053785186'],
['0.1725387909809054', '0.2501809549636665', '0.1892469865008623'],
['0.2316412813873359', '0.2992177107323640', '0.2598152855428822'],
['0.3187799923163431', '0.3334002478394827', '0.2139912936020207'],
['0.3710419941258571', '0.2507878307660951', '0.1774350085871969'],
['0.3116739096297613', '0.2018326705883265', '0.1062035992935087'],
['0.2260888649895278', '0.1658511121931230', '0.1557966256496020'],
['0.2307329565760341', '0.1946563172861149', '0.2560560427560625'],
['0.3212537354842268', '0.2446926242461327', '0.2699526655651979'],
['0.3190658637102883', '0.1639798582829078', '0.2033479700718845'],
['0.2639291422497501', '0.4063841070792547', '0.0757931284172781'],
['0.2259538405414187', '0.4014860963785403', '0.1123813341812155'],
['0.3045074368123725', '0.3737975048897033', '0.0661139711737767'],
['0.2613230342988596', '0.4437529298972389', '0.0489144810044546']
])

clear_Ni_MOCOM_opt = mult24([
['0.2722641601766250', '0.2529662041572345', '0.1822217115395579'],
['0.2267462552596598', '0.3389380301262131', '0.1648660277028081'],
['0.3157838804875621', '0.3074324570299595', '0.1121937016454754'],
['0.2243935798410539', '0.2546573008353110', '0.1000655266283262'],
['0.1765337792569313', '0.2526738398123435', '0.1890878084032278'],
['0.2321770798722959', '0.2976244577529907', '0.2574489974159138'],
['0.3181676856500327', '0.3293066397913347', '0.2145577086212941'],
['0.3682050223099776', '0.2497420186171965', '0.1771585756527575'],
['0.3119363264656582', '0.2049502364574571', '0.1092743357350494'],
['0.2276801384650838', '0.1718868969570273', '0.1552419614068016'],
['0.2315021469546747', '0.1955546358263050', '0.2522602694463135'],
['0.3181893287526825', '0.2435625329492326', '0.2658814287219978'],
['0.3153794831572942', '0.1659076378157896', '0.2029449091461648'],
['0.2626312501102042', '0.4035943544895452', '0.0755545227100918'],
['0.2258118803912443', '0.3992111992318690', '0.1136102735273178'],
['0.3041300602339393', '0.3714949605863472', '0.0673435931964058'],
['0.2577987759484085', '0.4389145142305136', '0.0463690651671732']
])

clear_Cu_MO_opt = mult24([
['0.2747807870446686', '0.2471563513959867', '0.1837722381269867'],
['0.2310169834665820', '0.3346263506051574', '0.1624402534350083'],
['0.3186184597414526', '0.3020980275417990', '0.1128960747668215'],
['0.2272100860991875', '0.2524707522973210', '0.0961049577724909'],
['0.1750713370632351', '0.2481023255694040', '0.1874163683003743'],
['0.2339207723116749', '0.2956784423760451', '0.2607533006054931'],
['0.3229083983787460', '0.3288699811943302', '0.2147168965509431'],
['0.3752104585171151', '0.2445414680092674', '0.1805632325743257'],
['0.3162455477839986', '0.1973491993407519', '0.1070024112845271'],
['0.2277243298082775', '0.1638914020242618', '0.1529316025039315'],
['0.2316372951920264', '0.1907184188984921', '0.2549147288196278'],
['0.3226971914194149', '0.2407967213813439', '0.2720002010073783'],
['0.3187813310852182', '0.1594762860671676', '0.2052444527139272'],
['0.1643344346273529', '0.4564950490932358', '0.1209068936351492'],
['0.1895276657040944', '0.4254867033578968', '0.1496288268671980'],
['0.1393332550902879', '0.4872550208475386', '0.0922775610358202']
])

clear_Cu_MCM_opt = mult24([
['0.2707678623483407', '0.2531460150367211', '0.1839614062256429'],
['0.2209362055699446', '0.3395087020766235', '0.1726128918550359'],
['0.3139344776774461', '0.3103916719359212', '0.1128265459796944'],
['0.2220606774122837', '0.2547990165549794', '0.0961186541785464'],
['0.1699036335275685', '0.2495398755049851', '0.1878812952173748'],
['0.2325177943936809', '0.2994922796432263', '0.2656422231661617'],
['0.3206772800101486', '0.3332662907116523', '0.2184770344453090'],
['0.3719377491387891', '0.2542684385843783', '0.1788963681512113'],
['0.3111658443222851', '0.2046047059604945', '0.1038489619841590'],
['0.2233981547335583', '0.1692137252394353', '0.1522291383821198'],
['0.2295202244363803', '0.1949403629539158', '0.2529163933832988'],
['0.3201664602656601', '0.2445260170619982', '0.2712655622145883'],
['0.3160701298439260', '0.1667858689443795', '0.1999981052962237'],
['0.2771121950411294', '0.3759812939229722', '0.0863210341947285'],
['0.2334682870559318', '0.3932448055548179', '0.1113391257110837'],
['0.2938828605240199', '0.4065282354152800', '0.0424909312040386'],
['0.2674430803655666', '0.4378860282315522', '0.0383701617441237']
])

clear_Cu_MOCM_H_n_bond_opt = mult24([
['0.2740218122097128', '0.2530129112468232', '0.1818566659323823'],
['0.2276207607121147', '0.3412220931087893', '0.1597305762557280'],
['0.3294829422472466', '0.2995365688052306', '0.1113517769173377'],
['0.2350614935039428', '0.2473209224615585', '0.0883615581829800'],
['0.1724908312462426', '0.2593653196130314', '0.1838383613795875'],
['0.2314853422195579', '0.3071537603038505', '0.2554146496700647'],
['0.3355654821414146', '0.3260695450349911', '0.2157908642903713'],
['0.3758351667761805', '0.2404385401995199', '0.1816895920216995'],
['0.3178107150034633', '0.1944683436978457', '0.1098441314365003'],
['0.2175274693337822', '0.1735444436502440', '0.1524648138789654'],
['0.2237168799516711', '0.2019352553867440', '0.2517638083213119'],
['0.3133846785754686', '0.2493298440927963', '0.2752623424526960'],
['0.3108559402978212', '0.1641541963836504', '0.2078333213857969'],
['0.2986913929376544', '0.3641018932624857', '0.0769129718843073'],
['0.2542666761043594', '0.3858923940043856', '0.0967856605179012'],
['0.3206201702323678', '0.3897910298781138', '0.0322175658297416'],
['0.2964593298403226', '0.4218566888699362', '0.0229434229759749']
])

clear_Cu_MOC__opt = mult24([
['0.2722368170396506', '0.2512199129442185', '0.1839612046303385'],
['0.2260286573505938', '0.3367218809473678', '0.1673437291957450'],
['0.3111357697290515', '0.3097814403925137', '0.1147474579187836'],
['0.2192462247446020', '0.2523014349351126', '0.0980782398705104'],
['0.1711673565767158', '0.2502933195666034', '0.1924021509452694'],
['0.2301709389443370', '0.2967173320407894', '0.2641070601389371'],
['0.3202740980369458', '0.3311233222413465', '0.2210788891200664'],
['0.3733586334200842', '0.2518694014455941', '0.1753237949550054'],
['0.3142849755019226', '0.2053631615089569', '0.1035996431654422'],
['0.2257943910073133', '0.1670520584887115', '0.1516440109580351'],
['0.2333692809030778', '0.1919049982632732', '0.2528581614420239'],
['0.3235327555773346', '0.2428204825966724', '0.2708338068181066'],
['0.3179765101326829', '0.1652789363590999', '0.2004195960191137'],
['0.2598875129372581', '0.3800375572608846', '0.1004552153972141'],
['0.2182755613011761', '0.4084546662076146', '0.1135747932038170'],
['0.2979863501305812', '0.3818367614679118', '0.0649205795549308']
])

clear_Cu_MOCM_opt = mult24([
['0.2629961305545777', '0.2598245994003558', '0.1812252256768028'],
['0.2036774236729218', '0.3401364073586249', '0.1905311887647357'],
['0.2730029208630165', '0.3376328970330993', '0.1031282588467369'],
['0.1907492969920126', '0.2639714585351515', '0.1119639171894727'],
['0.1735458431230240', '0.2378973248271515', '0.2168646573082958'],
['0.2390665892799703', '0.2941337858287384', '0.2735090291624857'],
['0.3065721671748055', '0.3470148005490030', '0.2125292684967991'],
['0.3540207039127780', '0.2822729259138266', '0.1445216037873910'],
['0.2862397106216865', '0.2256434958616638', '0.0877587408387411'],
['0.2191720709418297', '0.1746351522844597', '0.1499659071799157'],
['0.2588229939733724', '0.1896569793466091', '0.2509701200384173'],
['0.3389939533269369', '0.2577062375645914', '0.2454211854847998'],
['0.3266278578925670', '0.1847654666450436', '0.1704089918259583'],
['0.3491899608715018', '0.3431259430692442', '0.0875031621593625'],
['0.1948261804077994', '0.3431691280192294', '0.1117332102051696'],
['0.3896466130578612', '0.3605938144298744', '0.0653030330349533']
])

clear_Ni_MO_opt = mult24([
['0.2730334986331856', '0.2508921907149050', '0.1830495673775171'],
['0.2304254025647446', '0.3361211588398835', '0.1627505928460298'],
['0.3156182742527905', '0.3026834184804287', '0.1140981724231319'],
['0.2280434475646363', '0.2555250446334492', '0.0984406222441443'],
['0.1776496529006288', '0.2509022322043192', '0.1873441186016960'],
['0.2331566344248873', '0.2959375919442176', '0.2585119697286370'],
['0.3187494462986445', '0.3280372654803740', '0.2137590856741332'],
['0.3683759525086304', '0.2466741779811359', '0.1798394495681642'],
['0.3145261270361845', '0.2011317912209701', '0.1085405177411427'],
['0.2301030326191950', '0.1712857864887715', '0.1539752705112385'],
['0.2323473227229070', '0.1947367026183250', '0.2522870164921470'],
['0.3192437261677390', '0.2429355177948005', '0.2682163723283607'],
['0.3162091364300212', '0.1643951535388645', '0.2045430005500234'],
['0.1704496037371968', '0.4445841278340336', '0.1226217273796579'],
['0.1955274695715976', '0.4076985913286363', '0.1434986414600058'],
['0.1455596059003575', '0.4814717488968775', '0.1020938750739596']

])

clear_Ni_MCM_opt = mult24([
['0.2716052925784512', '0.2530264044734258', '0.1834413592746409'],
['0.2229866944468610', '0.3386186328008286', '0.1683430407527373'],
['0.3093343937002386', '0.3120347732157153', '0.1141468919274850'],
['0.2234350808231386', '0.2561875555798442', '0.1009718523195429'],
['0.1754935129461185', '0.2511375989635536', '0.1906373236793656'],
['0.2318326293796885', '0.2953228504088835', '0.2598451414114219'],
['0.3175536980479684', '0.3289159675164883', '0.2170099918029710'],
['0.3680427059777213', '0.2521086844283733', '0.1777674369771349'],
['0.3109754781012351', '0.2075402930834445', '0.1083313713187315'],
['0.2272901073604187', '0.1727809853444444', '0.1546528615693206'],
['0.2308742559865563', '0.1940218785423870', '0.2532643329033851'],
['0.3182639048912558', '0.2429304554007783', '0.2669679442996438'],
['0.3148792011461055', '0.1676885273379063', '0.2021227720627380'],
['0.2765799426248736', '0.3765138141497566', '0.0865612113731280'],
['0.2327626442082512', '0.3950239556502009', '0.1111785989737131'],
['0.2943998159858240', '0.4062563776446622', '0.0424017293283703'],
['0.2686535584619536', '0.4380145787926400', '0.0375519733590104']
])


clear_Ni_MOCM_H_n_bond_opt = mult24([
['0.2735469202228711', '0.2554115146126319', '0.1771653598014718'],
['0.2302054557943378', '0.3420179299579092', '0.1510394550368659'],
['0.3192608714265476', '0.3028854157249776', '0.1054510959690101'],
['0.2272698163454194', '0.2548987508657965', '0.0942654362283146'],
['0.1772908070363346', '0.2570251456968763', '0.1811982602821395'],
['0.2329994373607292', '0.3045561178733973', '0.2481900020030482'],
['0.3195060285966601', '0.3334510734032800', '0.2049641509463864'],
['0.3698164842290180', '0.2498126330993098', '0.1788685128640458'],
['0.3122419365575361', '0.2003426952882039', '0.1095903975323456'],
['0.2269054907651381', '0.1738990515235590', '0.1553198485690471'],
['0.2282921325444394', '0.2047149545329642', '0.2514088255419241'],
['0.3168902480091786', '0.2512754335606512', '0.2640009824387046'],
['0.3132322870549500', '0.1698780985926941', '0.2062062001540351'],
['0.3540113638273493', '0.3361444526807239', '0.0518366383243857'],
['0.2147717216172833', '0.4058527792321422', '0.1184144868355510'],
['0.3768760408907211', '0.3595418906428651', '0.0158425329302604'],
['0.2417800410548012', '0.4174858127120071', '0.0902998978757964']   
])

#otrzymana energia jest OG ROM NA
clear_Ni_MOC__opt = mult24([
['0.2773915318837867', '0.2431032655568980', '0.1844059454892380'],
['0.2389447520355296', '0.3321019860994412', '0.1635682161652840'],
['0.3130724472988961', '0.3174108226400625', '0.1056805013415748'],
['0.2285605239769603', '0.2556055321082836', '0.1021570981449260'],
['0.1807858040280415', '0.2506064566969425', '0.1884374774279849'],
['0.2356150472439307', '0.2929510107469016', '0.2573307355600857'],
['0.3204727365581848', '0.3230128401645534', '0.2158011916518380'],
['0.3633982542927782', '0.2488612755070736', '0.1792757870040639'],
['0.3104067230351601', '0.2075288689690682', '0.1137273458006828'],
['0.2301416594167177', '0.1774964250391792', '0.1542963157014788'],
['0.2323241712706006', '0.1999865818130719', '0.2485508267579226'],
['0.3169967796408608', '0.2446078036539735', '0.2629807055829792'],
['0.3130299776832849', '0.1694980053807542', '0.2014348844670278'],
['0.2628281371278708', '0.3698778494113343', '0.1018680732327485'],
['0.2190325867259020', '0.4127273867332818', '0.1054320354797000'],
['0.2717247011148219', '0.3774005561458516', '0.0904011935258041']
])

clear_Ni_MOCM_opt= mult24([
['0.2710890571778696', '0.2557178605399997', '0.1797932425213735'],
['0.2213917835383397', '0.3411840063317034', '0.1649963183341402'],
['0.3086885816125514', '0.3117241949597230', '0.1101854518196411'],
['0.2217205539997919', '0.2573270879516730', '0.0991427132372904'],
['0.1751816487100828', '0.2499667309857952', '0.1899774342782412'],
['0.2328278256410243', '0.2960845905336993', '0.2576683814880235'],
['0.3168011513949415', '0.3335216899647466', '0.2107730868054707'],
['0.3678716704070267', '0.2545346317984180', '0.1743765918801925'],
['0.3111050603457111', '0.2075809260363922', '0.1069745697171839'],
['0.2285391676439589', '0.1734808025077434', '0.1541067699638219'],
['0.2348068222176616', '0.1955742191902325', '0.2515875802712900'],
['0.3197576135649278', '0.2463814620891006', '0.2623532827632390'],
['0.3180879443746373', '0.1687124382464718', '0.1994826899561356'],
['0.3240221753724549', '0.3598637322149157', '0.0584219069971397'],
['0.1828584717015603', '0.3973674476597115', '0.1593916504765353'],
['0.3324008889641219', '0.3931585956563401', '0.0241058294902975']
])

## Generowanie listy wszystkich możliwości do przypisywania klastrów Elki

In [ ]:
def rotation_to_cuni(nr_klastra, rotacja):
    
    topologia = topology_to_list(df['topologia'][nr_klastra])
#     print(topologia)
    pozycje_niklu = []
    wynik = []
    for i in range(len(topologia)):
        if topologia[i] == 'Ni':
            pozycje_niklu.append(i)
    for pos in rotacja:        
        if pos in pozycje_niklu:
            wynik.append('n') 
        else:
            wynik.append('c') 
#ostatnim krokiem będzie sprawdzanie czy w wyniku takiej rotacji nie powstał klaster, który
#jest bezrotacyjnym innym klastrem o takiej samej liczbie nikli
#można to sprawdzić po prostu porównując wszystkie topologie
#co bardzo ciekawe testy wskazały, że żadna rotacja nie powtarza się w ten sposób
#żadna rotacja też nie jest taka sama jak już wygenerowana rotacja danego rodzaju
    
    licznik = 0
    for cluster in range(len(df)):
        if str(wynik) == df['topologia'][cluster]:
            return 'w wyniku rotacji ' +str(rotacja[1:3]) +' otrzymano topologie klastra nr ' + str(cluster)
            break
        else:
            licznik +=1
    
    if licznik == 192:
        
        return 'w wyniku rotacji ' +str(rotacja[1:3]) + ' klastra ' + str(nr_klastra) + ' powstał nowy układ   ' + str(' '.join(wynik)), wynik
    
def all_non_repetitive_rotations(nr_klastra, edges, m_edges, corners):
    
    possible_rotation = rotacja(nr_klastra)
    all_new_clusters = []
    all_new_corners_clusters = []
    all_new_edges_clusters = []
    all_new_m_edges_clusters = []
    
    if corners == 1:
        print()
        print('narożnikowe przyłączenia')
        print()
        corner_rotation = possible_rotation[2]
        for i in range(len(corner_rotation)):
            cluster = rotation_to_cuni(nr_klastra, corner_rotation[i])
            if len(cluster) == 2:
                print(cluster[0])
                if cluster[1] not in all_new_corners_clusters:
                    all_new_corners_clusters.append(cluster[1])
            else:
                pass
        all_new_clusters.append(all_new_corners_clusters)
                
    if edges == 1:
        print()
        print('krawędziowe przyłączenia')
        print()
        edge_rotation = possible_rotation[0]
        for i in range(len(edge_rotation)):
            cluster = rotation_to_cuni(nr_klastra, edge_rotation[i])
            if len(cluster) == 2:
                print(cluster[0])
                if cluster[1] not in all_new_edges_clusters:
                    all_new_edges_clusters.append(cluster[1])
            else:
                pass
        all_new_clusters.append(all_new_edges_clusters)
    if m_edges == 1:
        print()
        print('krawędziowe lustrzane przyłączenia')
        print()
        m_edge_rotation = possible_rotation[1]
        for i in range(len(m_edge_rotation)):
            cluster = rotation_to_cuni(nr_klastra, m_edge_rotation[i])
            if len(cluster) == 2:
                print(cluster[0])
                if cluster[1] not in all_new_m_edges_clusters:
                    all_new_m_edges_clusters.append(cluster[1])
            else:
                pass
        all_new_clusters.append(all_new_m_edges_clusters)
        return ((all_new_clusters))
    


results = []
for i in range(192):
    lista = (all_non_repetitive_rotations(i,1,1,1))
    lenght = len(lista[0])+len(lista[1])+len(lista[2])
    original_lenght = df['C/O/M-O-C/M-O-C-_'][i]+2*df['M-O-C-O/M-C-M'][i]
    
    
    if original_lenght>lenght+3:
        results.append('o ' + str(i))
    else:
        results.append('nic się nie powtarza: ' + str(i))
for char in range(len(df)):
    print('w wyniku rotacji [1,2] klastra ' + str(df['Unnamed: 0'][char]) + ' powstał stary układ   ' + str(' '.join(topology_to_list(df['topologia'][char]))))